### RAG와 LLM을 활용한 질의응답 실습
- LLM API 호출 시 RAG를 통해 연관 정보 추출 후 Prompt에 추가하는 실습
- RAG 추가 전후로 평가 진행

In [2]:
import pickle

with open('./res/rag_data.pkl', 'rb') as f:
    rag_data = pickle.load(f)

In [8]:
rag_data.keys()

dict_keys(['questions', 'contexts', 'contexts_answer_idx', 'contexts_answers', 'answers'])

In [9]:
questions = rag_data['questions'][:10]
contexts = rag_data['contexts'][:10]
answers = rag_data['answers'][:10]
len(questions)

10

In [15]:
print(questions[0])
print(contexts[0])
print(answers[0])

글로벌 저금리 현상이 부각된 원인은 무엇인가요?
['... 변화의 동인은 크게 두 가지로 분류할\n수 있다. 하나는 금융업계 전반에 부각되는 디지털화(digitalization)1)이고\n다른 하나는 저금리·저성장으로 대표되는 뉴노멀(new normal)이다.  ... . 뉴노멀의 특징으로는 금융투자\n및 자산운용의 어려움이 가중되는 경향, 고수익-고위험 부문으로의 쏠림\n현상 등의 위험요인과 함께 자산관리서비스 수요의 증가 등 대응 노력을\n살펴본다. 제III장은 디지털화 및 뉴노멀에 대응하는 해외 사례들을 살펴본다.  ... 뉴노멀의\n주요 특징인 저성장, 저금리 등은 일본의 금융투자업계가 30년 전부터 고민해\n온 문제이기 때문이다.  ... 제III장에서 살펴본 해외\n사례들에서 살펴본 금융투자업계 변화의 규칙성이 어느 정도 유지될 것이라는\n1 디지털화(digitalization)에 대한 ...', '3. 문제점\n1) 기업신용위험 평가항목의 적정성\n글로벌 금융위기 이후 저금리 기조가 지속되었고, 최근 추가적으로\n금리가 하락함에 따라 이자보상배율 등의 지표들이 기업의 부실을\n측정하는 데 적정한 지표인지에 대한 검토가 필요\n기업신용위험 정 ... 작용할 것으로 예상\n저금리 기조가 지속되는 가운데 금융비용이 감소하여 이자보상배\n율이 기업의 부실을 가늠하는 지표로 적합한지에 대해 의문이\n발생\n더불어 코로나19 사태로 인한 중소기업에 대한 금융권 대출 만기\n연장 및 이자상환 유예조치로 이 ... 는 추세\n이러한 졸업비율 감소 현상의 원인으로 저조한 경기의 영향이\n있었던 것으로 추정되지만 개시연도를 기준으로 졸 ...', '2. 뉴노멀의 도래(신 3저의 도래)\n최근 코로나 사태를 포함하여 2008년 글로벌 금융위기 이후 전세계적인\n저금리·저성장 상황\n∙ 글로벌 금융위기를 극복하는 과정에서 미국, 유럽, 일본 등 주요국의\n완화적 통화정책으로 글로벌 저금리 현상이 부각\n∙ 코로나 팬데믹이 진정되더라도 글로벌 저금리 추세가 지속되는 가운

### 평가
- RAGAS의 Answer Correctedness 평가 지표
  - 모델의 답변과 실제 정답 간의 일치율 측정

### Baseline (w/o RAG)

In [ ]:
from tqdm import tqdm

from utils import call_openai, get_embeddings, cosine_similarity

predictions = []
for i in tqdm(range(len(questions))):
    prompt = f"""You are an expert in Finance. Please answer to the question given below.

Question:
{questions[i]}
"""

    prediction = call_openai(prompt, model='gpt-4o-2024-05-13')
    predictions.append(prediction)

100%|██████████| 10/10 [01:21<00:00,  8.18s/it]


In [ ]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import answer_correctness


data_samples = {
    'question': questions,
    'answer': predictions,
    'ground_truth': answers
}

dataset = Dataset.from_dict(data_samples)

score = evaluate(dataset, metrics=[answer_correctness])
score

/Users/rein20/Documents/lectures/ai-service/retrieval-augmented-generation/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Evaluating: 100%|██████████| 10/10 [01:17<00:00,  7.73s/it]


{'answer_correctness': 0.4985}

In [5]:
print(questions[0])

글로벌 저금리 현상이 부각된 원인은 무엇인가요?


In [6]:
print(predictions[0])

글로벌 저금리 현상이 부각된 원인에는 여러 가지 요인이 복합적으로 작용하고 있습니다. 주요 원인들을 살펴보면 다음과 같습니다:

1. **경제 성장 둔화**: 선진국을 중심으로 경제 성장률이 둔화되면서 중앙은행들이 경기 부양을 위해 금리를 낮추는 정책을 채택하게 되었습니다. 이는 저금리 현상의 주요 원인 중 하나입니다.

2. **인구 고령화**: 많은 선진국에서 인구 고령화가 진행되면서 소비와 투자가 줄어들고, 저축이 증가하는 경향이 있습니다. 이는 금리 하락을 유도하는 요인으로 작용합니다.

3. **글로벌 금융 위기**: 2008년 글로벌 금융 위기 이후, 각국 중앙은행들은 경제 회복을 위해 금리를 낮추고 양적 완화 정책을 시행했습니다. 이러한 정책들은 장기적으로 저금리 환경을 조성하는 데 기여했습니다.

4. **디플레이션 압력**: 일부 국가에서는 디플레이션(물가 하락) 압력이 존재하여 중앙은행들이 금리를 낮추는 정책을 통해 물가 상승을 유도하려고 합니다.

5. **국제 자본 이동**: 글로벌화로 인해 자본이 국경을 넘어 자유롭게 이동하면서, 금리가 낮은 국가로 자본이 유입되는 현상이 발생합니다. 이는 금리 하락을 더욱 촉진시킵니다.

6. **기술 발전**: 기술 발전으로 인해 생산성이 향상되고, 이는 물가 상승 압력을 억제하는 요인으로 작용합니다. 중앙은행들은 이러한 상황에서 금리를 낮추는 경향이 있습니다.

7. **정책적 요인**: 각국 정부와 중앙은행들이 경제 안정과 성장을 위해 저금리 정책을 지속적으로 유지하는 경우도 있습니다. 이는 경기 부양과 고용 창출을 목표로 하는 정책적 결정입니다.

이와 같은 여러 요인들이 복합적으로 작용하여 글로벌 저금리 현상이 부각되고 있습니다.


### LLM + RAG
- text-embedding-3-large 임베딩 모델 사용

In [16]:
import numpy as np
from tqdm import tqdm

from utils import call_openai, get_embeddings, cosine_similarity


def retrieve_context(question, contexts):
    question_embedding = get_embeddings([question], model='text-embedding-3-large')[0]
    context_embeddings = get_embeddings(contexts, model='text-embedding-3-large')

    similarities = [cosine_similarity(question_embedding, context_embedding) for context_embedding in context_embeddings]

    most_relevant_index = np.argmax(similarities)
    return contexts[most_relevant_index]

predictions = []
for i in tqdm(range(len(questions))):
    context = retrieve_context(questions[i], contexts[i])
    prompt = f"""You are an expert in Finance. Please answer to the question given below. Use information given in Context appropriately.

Context:
{context}

Question:
{questions[i]}
"""
    prediction = call_openai(prompt, model='gpt-4o-2024-05-13')
    predictions.append(prediction)

100%|██████████| 10/10 [00:42<00:00,  4.28s/it]


In [18]:
!pip install ragas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.6/303.6 kB 6.3 MB/s eta 0:00:00a 0:00:01
  Using cached tiktoken-0.12.0-cp311-cp311-macosx_11_0_arm64.whl (995 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.9 MB/s eta 0:00:00
  Using cached typer-0.19.2-py3-none-any.whl (46 kB)
  Using cached rich-14.1.0-py3-none-any.whl (243 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.5/155.5 kB 8.4 MB/s eta 0:00:00
  Using cached gitpython-3.1.45-py3-none-any.whl (208 kB)
  Using cached pillow-11.3.0-cp311-cp311-macosx_11_0_arm64.whl (4.7 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 10.6 MB/s eta 0:00:00a 0:00:01
  Using cached langchain-0.3.27-py3-none-any.whl (1.0 MB)
  Using cached langchain_core-0.3.78-py3-none-any.whl (449 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 10.5 MB/s eta 0:00:0000:0100:01
  Using cached langchain_openai-0.3.35-py3-none-any.w

In [ ]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import answer_correctness

data_samples = {
    'question': questions,
    'answer': predictions,
    'ground_truth': answers
}

dataset = Dataset.from_dict(data_samples)

score = evaluate(dataset, metrics=[answer_correctness])
score

ImportError: cannot import name 'context_relevancy' from 'ragas.metrics' (/Users/sally/Workspace/AI-SERVICE/rag_practical/.venv/lib/python3.11/site-packages/ragas/metrics/__init__.py)

In [9]:
print(predictions[0])

글로벌 저금리 현상이 부각된 원인은 여러 가지가 있습니다. 주요 원인으로는 다음과 같은 요소들이 있습니다:

1. **완화적 통화정책**: 2008년 글로벌 금융위기 이후 미국, 유럽, 일본 등 주요국들이 경제를 회복시키기 위해 완화적 통화정책을 시행하였습니다. 이러한 정책은 금리를 낮추고 유동성을 공급하여 경제를 부양하려는 목적이 있었습니다.

2. **경제구조적 요인**: 글로벌 금융위기 이전부터 이어진 경제구조적 요인들도 저금리 현상에 기여하였습니다. 예를 들어, 선진국의 성장세 둔화와 신흥국의 성장으로 인해 글로벌 무역 불균형이 심화되고, 저축과잉현상(global savings glut)이 발생하였습니다.

3. **코로나 팬데믹**: 최근 코로나 사태로 인해 경제가 큰 타격을 받으면서, 각국 중앙은행들은 다시 한 번 완화적 통화정책을 강화하였습니다. 이로 인해 저금리 기조가 더욱 부각되었습니다.

이러한 요인들이 복합적으로 작용하여 글로벌 저금리 현상이 지속되고 있습니다.


In [10]:
print(answers[0])

글로벌 저금리 현상이 부각된 원인은 여러 가지입니다. 첫째, 2008년 글로벌 금융위기를 극복하는 과정에서 미국, 유럽, 일본 등 주요국의 완화적 통화정책이 있습니다. 둘째, 선진국의 성장세가 둔화되고, 신흥국이 성장하면서 글로벌 무역 불균형이 심화되고 저축과잉현상이 발생했습니다. 셋째, 기술발전에 따른 자본재 가격 하락과 신기술 혁신에 따른 수익률이 낮아져 경제 전체적인 유효투자가 감소했습니다. 넷째, 세계적인 인구증가율 감소 추세는 유효수요 및 자본의 한계생산성과 투자이익 및 투자증가율을 감소시켰습니다. 따라서 이러한 요인들이 복합적으로 작용하여 글로벌 저금리 현상이 부각되었습니다.
